In [5]:
from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model

from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.callbacks import EarlyStopping
import random
import tensorflow as tf
from lightgbm import LGBMRegressor

curr_dir = os.getcwd()

In [12]:
max_depth = 5
n_estimators = 400             
lr = 0.01
num_leaves = 31
early_stopping_rounds = 100     
verbose = True  

In [22]:
# es = EarlyStopping(monitor='val_mse', verbose=1, patience=10)

# # Set Directory
# save_dir = os.path.join(os.getcwd(),'save')

# model_name = 'lgbm_v5.{epoch:02d}.h5' 
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# filepath = os.path.join(curr_dir+"\\save\\"+model_name)


# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_mse',
#                              verbose=1,
#                              save_best_only=True)

# callbacks = [checkpoint,es]

In [23]:
def model_train(group_number, curr_dir):
 
    path_train = os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\train")
    path_test = os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\test")
    path_val = os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\validation")

    folders_tr = os.listdir(path_train)
    folders_test = os.listdir(path_test)
    folders_val = os.listdir(path_val)
 
    train = pd.DataFrame()
    for files in folders_tr:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\train\\")+files)
        train = pd.concat([train, df])            
              
    test = pd.DataFrame()
    for files in folders_test:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\test\\")+files)
        test = pd.concat([test, df])   
            
    validation = pd.DataFrame()
    for files in folders_val:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\validation\\")+files)
        validation = pd.concat([validation, df]) 
    
    train['NDVI']=(train['Band4']-train['Band3'])/(train['Band4']+train['Band3']+3) 
    train['NDWI']=(train['Band6']-train['Band2'])/(train['Band6']+train['Band2']+3) 
    
    test['NDVI']=(test['Band4']-test['Band3'])/(test['Band4']+test['Band3']+3) 
    test['NDWI']=(test['Band6']-test['Band2'])/(test['Band6']+test['Band2']+3) 
    
    validation['NDVI']=(validation['Band4']-validation['Band3'])/(validation['Band4']+validation['Band3']+3) 
    validation['NDWI']=(validation['Band6']-validation['Band2'])/(validation['Band6']+validation['Band2']+3) 
        
# train_ta 
    X_train_ta = train.drop(columns = ['YearMonthDayHourMinute','STN', 'Lon', 'Lat', 'isitu-LST', 'insitu-TA'])
    X_test_ta = test.drop(columns = ['YearMonthDayHourMinute','STN', 'Lon', 'Lat', 'isitu-LST', 'insitu-TA'])
    y_train_ta=train[['insitu-TA']].values
    y_test_ta=test[['insitu-TA']].values
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves).fit(X_train_ta, y_train_ta,\
              eval_set=[(X_test_ta, y_test_ta)],\
              eval_metric=["mse"],\
              early_stopping_rounds=early_stopping_rounds,\
              verbose=verbose) 
    
    # filename = 'lgbm_modelv5.model'
    # lgbm_modelv5.save_model(filename) #save
    # lgbm_modelv5 = xgb.XGBRegressor() # 모델 초기화
    # lgbm_modelv5.load_model(filename) # 모델 불러오기
    
    # feature_importance = lgbm_modelv5.feature_importances_
    # sorted_idx = np.argsort(feature_importance)
    # fig = plt.figure(figsize=(12, 6))
    # plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
    # plt.yticks(range(len(sorted_idx)), np.array(X_test_ta.columns)[sorted_idx])
    # plt.title('Feature Importance')
    # plt.show()
    #     Prediction TA    
#     model = load.model(os.listdir(os.path.join(curr_dir+"\\save"))[-1])
    
    validation_ = validation.drop(columns = ['YearMonthDayHourMinute','STN', 'Lon', 'Lat'])
    prediction = model.predict(validation_)
    validation_partial = validation[['YearMonthDayHourMinute', 'STN']]
    validation_partial = validation_partial.assign(TA = prediction)
    
    
    
    prediction_train = model.predict(X_train_ta)
    X_train_ta['ta'] = prediction_train
    
    prediction_test = model.predict(X_test_ta)
    X_test_ta['ta'] = prediction_test
    
    validation_['ta'] = prediction
    
# train_lst 
    X_train_lst = X_train_ta 
    X_test_lst = X_test_ta
    
    y_train_lst=train[['isitu-LST']].values
    y_test_lst=test[['isitu-LST']].values
    
    model = LGBMRegressor(objective= "regression",\
                          max_depth= max_depth,\
                          n_estimators= n_estimators,\
                          learning_rate= lr,\
                          num_leaves = num_leaves).fit(X_train_lst, y_train_lst,\
              eval_set=[(X_test_lst, y_test_lst)],\
              eval_metric=["mse"],\
              early_stopping_rounds=early_stopping_rounds,\
              verbose=verbose) 
    # filename = 'lgbm_modelv5.model'
    # lgbm_modelv5.save_model(filename) #save
    # lgbm_modelv5 = xgb.XGBRegressor() # 모델 초기화
    # lgbm_modelv5.load_model(filename) # 모델 불러오기
    
    # feature_importance = lgbm_modelv5.feature_importances_
    # sorted_idx = np.argsort(feature_importance)
    # fig = plt.figure(figsize=(12, 6))
    # plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
    # plt.yticks(range(len(sorted_idx)), np.array(X_test_lst.columns)[sorted_idx])
    # plt.title('Feature Importance')
    # plt.show()
#     model = load.model(os.listdir(os.path.join(curr_dir+"\\save"))[-1])
    
    prediction = model.predict(validation_)
    validation_partial = validation_partial.assign(LST = prediction)
    validation_partial = validation_partial[['YearMonthDayHourMinute','STN','LST', 'TA']]
    validation_partial.to_csv("lgbm_group"+str(group_number)+"_v5.csv", index = False)

In [24]:
with tf.device('/CPU:0'):

    for i in range(1, 8):
        model_train(i, curr_dir)

C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l2: 70.0065
[2]	valid_0's l2: 68.717
[3]	valid_0's l2: 67.453
[4]	valid_0's l2: 66.214
[5]	valid_0's l2: 64.9994
[6]	valid_0's l2: 63.8147
[7]	valid_0's l2: 62.6439
[8]	valid_0's l2: 61.5075
[9]	valid_0's l2: 60.3904
[10]	valid_0's l2: 59.2921
[11]	valid_0's l2: 58.2181
[12]	valid_0's l2: 57.1629
[13]	valid_0's l2: 56.1268
[14]	valid_0's l2: 55.1148
[15]	valid_0's l2: 54.1251
[16]	valid_0's l2: 53.1421
[17]	valid_0's l2: 52.1819
[18]	valid_0's l2: 51.2474
[19]	valid_0's l2: 50.3261
[20]	valid_0's l2: 49.4277
[21]	valid_0's l2: 48.5396
[22]	valid_0's l2: 47.6702
[23]	valid_0's l2: 46.8255
[24]	valid_0's l2: 45.9872
[25]	valid_0's l2: 45.1743
[26]	valid_0's l2: 44.3737
[27]	valid_0's l2: 43.5783
[28]	valid_0's l2: 42.8035
[29]	valid_0's l2: 42.0491
[30]	valid_0's l2: 41.3024
[31]	valid_0's l2: 40.572
[32]	valid_0's l2: 39.8637
[33]	valid_0's l2: 39.1628
[34]	valid_0's l2: 38.4785
[35]	valid_0's l2: 37.8077
[36]	valid_0's l2: 37.1501
[37]	valid_0's l2: 36.4958
[38]	valid_0's

[306]	valid_0's l2: 1.87705
[307]	valid_0's l2: 1.86896
[308]	valid_0's l2: 1.86165
[309]	valid_0's l2: 1.85324
[310]	valid_0's l2: 1.84421
[311]	valid_0's l2: 1.83705
[312]	valid_0's l2: 1.82872
[313]	valid_0's l2: 1.82183
[314]	valid_0's l2: 1.81359
[315]	valid_0's l2: 1.8062
[316]	valid_0's l2: 1.79795
[317]	valid_0's l2: 1.79097
[318]	valid_0's l2: 1.7843
[319]	valid_0's l2: 1.77738
[320]	valid_0's l2: 1.7694
[321]	valid_0's l2: 1.76256
[322]	valid_0's l2: 1.7561
[323]	valid_0's l2: 1.7488
[324]	valid_0's l2: 1.74266
[325]	valid_0's l2: 1.73575
[326]	valid_0's l2: 1.72872
[327]	valid_0's l2: 1.72232
[328]	valid_0's l2: 1.71503
[329]	valid_0's l2: 1.7086
[330]	valid_0's l2: 1.70208
[331]	valid_0's l2: 1.69543
[332]	valid_0's l2: 1.69045
[333]	valid_0's l2: 1.6854
[334]	valid_0's l2: 1.67886
[335]	valid_0's l2: 1.67318
[336]	valid_0's l2: 1.66707
[337]	valid_0's l2: 1.66062
[338]	valid_0's l2: 1.65498
[339]	valid_0's l2: 1.649
[340]	valid_0's l2: 1.6431
[341]	valid_0's l2: 1.6378
[34

C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l2: 98.3141
[2]	valid_0's l2: 96.3986
[3]	valid_0's l2: 94.5186
[4]	valid_0's l2: 92.6755
[5]	valid_0's l2: 90.8748
[6]	valid_0's l2: 89.1079
[7]	valid_0's l2: 87.3755
[8]	valid_0's l2: 85.676
[9]	valid_0's l2: 84.0182
[10]	valid_0's l2: 82.3879
[11]	valid_0's l2: 80.7932
[12]	valid_0's l2: 79.225
[13]	valid_0's l2: 77.7009
[14]	valid_0's l2: 76.1963
[15]	valid_0's l2: 74.7355
[16]	valid_0's l2: 73.2866
[17]	valid_0's l2: 71.8866
[18]	valid_0's l2: 70.5028
[19]	valid_0's l2: 69.1623
[20]	valid_0's l2: 67.8375
[21]	valid_0's l2: 66.5414
[22]	valid_0's l2: 65.272
[23]	valid_0's l2: 64.0176
[24]	valid_0's l2: 62.7951
[25]	valid_0's l2: 61.5987
[26]	valid_0's l2: 60.4306
[27]	valid_0's l2: 59.2721
[28]	valid_0's l2: 58.1415
[29]	valid_0's l2: 57.037
[30]	valid_0's l2: 55.9525
[31]	valid_0's l2: 54.8969
[32]	valid_0's l2: 53.8576
[33]	valid_0's l2: 52.842
[34]	valid_0's l2: 51.8461
[35]	valid_0's l2: 50.8646
[36]	valid_0's l2: 49.9027
[37]	valid_0's l2: 48.9589
[38]	valid_0's 

...

[1]	valid_0's l2: 80.7799
[2]	valid_0's l2: 79.1489
[3]	valid_0's l2: 77.5642
[4]	valid_0's l2: 76.0257
[5]	valid_0's l2: 74.494
[6]	valid_0's l2: 73.0161
[7]	valid_0's l2: 71.5624
[8]	valid_0's l2: 70.1154
[9]	valid_0's l2: 68.713
[10]	valid_0's l2: 67.3586
[11]	valid_0's l2: 66.0127
[12]	valid_0's l2: 64.6838
[13]	valid_0's l2: 63.4005
[14]	valid_0's l2: 62.1303
[15]	valid_0's l2: 60.9128
[16]	valid_0's l2: 59.6931
[17]	valid_0's l2: 58.5261
[18]	valid_0's l2: 57.3763
[19]	valid_0's l2: 56.2303
[20]	valid_0's l2: 55.1114
[21]	valid_0's l2: 54.0364
[22]	valid_0's l2: 52.9675
[23]	valid_0's l2: 51.9428
[24]	valid_0's l2: 50.9257
[25]	valid_0's l2: 49.9158
[26]	valid_0's l2: 48.961
[27]	valid_0's l2: 47.991
[28]	valid_0's l2: 47.0829
[29]	valid_0's l2: 46.1549
[30]	valid_0's l2: 45.2454
[31]	valid_0's l2: 44.3813
[32]	valid_0's l2: 43.4983
[33]	valid_0's l2: 42.6472
[34]	valid_0's l2: 41.8357
[35]	valid_0's l2: 41.0417
[36]	valid_0's l2: 40.2545
[37]	valid_0's l2: 39.4957
[38]	valid_0's

C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l2: 71.5861
[2]	valid_0's l2: 70.2517
[3]	valid_0's l2: 68.9455
[4]	valid_0's l2: 67.6613
[5]	valid_0's l2: 66.401
[6]	valid_0's l2: 65.1698
[7]	valid_0's l2: 63.9571
[8]	valid_0's l2: 62.7754
[9]	valid_0's l2: 61.5801
[10]	valid_0's l2: 60.4347
[11]	valid_0's l2: 59.2799
[12]	valid_0's l2: 58.186
[13]	valid_0's l2: 57.0815
[14]	valid_0's l2: 56.0005
[15]	valid_0's l2: 54.9404
[16]	valid_0's l2: 53.9059
[17]	valid_0's l2: 52.8943
[18]	valid_0's l2: 51.9016
[19]	valid_0's l2: 50.9051
[20]	valid_0's l2: 49.9581
[21]	valid_0's l2: 49.0116
[22]	valid_0's l2: 48.1098
[23]	valid_0's l2: 47.2157
[24]	valid_0's l2: 46.3072
[25]	valid_0's l2: 45.4209
[26]	valid_0's l2: 44.5824
[27]	valid_0's l2: 43.7252
[28]	valid_0's l2: 42.9178
[29]	valid_0's l2: 42.1254
[30]	valid_0's l2: 41.316
[31]	valid_0's l2: 40.5267
[32]	valid_0's l2: 39.7805
[33]	valid_0's l2: 39.0166
[34]	valid_0's l2: 38.3173
[35]	valid_0's l2: 37.5729
[36]	valid_0's l2: 36.8862
[37]	valid_0's l2: 36.1834
[38]	valid_0'

C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l2: 99.9698
[2]	valid_0's l2: 98.0279
[3]	valid_0's l2: 96.1043
[4]	valid_0's l2: 94.2432
[5]	valid_0's l2: 92.4092
[6]	valid_0's l2: 90.612
[7]	valid_0's l2: 88.8517
[8]	valid_0's l2: 87.1028
[9]	valid_0's l2: 85.4236
[10]	valid_0's l2: 83.7644
[11]	valid_0's l2: 82.1124
[12]	valid_0's l2: 80.5196
[13]	valid_0's l2: 78.9702
[14]	valid_0's l2: 77.4471
[15]	valid_0's l2: 75.9568
[16]	valid_0's l2: 74.4799
[17]	valid_0's l2: 73.0488
[18]	valid_0's l2: 71.6227
[19]	valid_0's l2: 70.2504
[20]	valid_0's l2: 68.8899
[21]	valid_0's l2: 67.5667
[22]	valid_0's l2: 66.2573
[23]	valid_0's l2: 64.9902
[24]	valid_0's l2: 63.7295
[25]	valid_0's l2: 62.5081
[26]	valid_0's l2: 61.3109
[27]	valid_0's l2: 60.1401
[28]	valid_0's l2: 58.9694
[29]	valid_0's l2: 57.8318
[30]	valid_0's l2: 56.7226
[31]	valid_0's l2: 55.6159
[32]	valid_0's l2: 54.5475
[33]	valid_0's l2: 53.4844
[34]	valid_0's l2: 52.4569
[35]	valid_0's l2: 51.4328
[36]	valid_0's l2: 50.4364
[37]	valid_0's l2: 49.4807
[38]	valid_